In [107]:
import os, numpy, PIL
from PIL import Image
import numpy as np
from numpy import newaxis
from time import sleep
import warnings
import pandas as pd
from numpy.linalg import inv

import scipy.linalg   # SciPy Linear Algebra Library

import re
warnings.filterwarnings("ignore")


In [108]:
PATH = '/home/maghareh/Desktop/ThirdSemester/ComputationalDataMining/HW3'

In [109]:
matrix_A = {'0':[],'1':[],'2':[],'3':[],'4':[],'5':[],'6':[],'7':[],'8':[],'9':[]}


for digit in range(0,10):
    allfiles=os.listdir(PATH+f'/TinyMNIST/train/{digit}/')
    imlist=[PATH+f'/TinyMNIST/train/{digit}/'+filename for filename in allfiles if  filename[-4:] in [".jpg",".JPG"]]

    # Assuming all images are the same size, get dimensions of first image
    w,h=Image.open(imlist[0]).size
    N=len(imlist)

    # Build up average pixel intensities, casting each image as an array of floats
    for im in imlist[0:100]:
        imarr=numpy.array(Image.open(im),dtype=numpy.float)
        imarr = imarr.flatten()
        
        matrix_A[str(digit)].append(imarr)
print(pd.DataFrame(matrix_A['0']))


    0    1    2    3    4    5    6    7    8     9    ...  774  775  776  \
0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0   0.0  ...  0.0  0.0  0.0   
1   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  5.0   0.0  ...  0.0  0.0  0.0   
2   0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   2.0  ...  0.0  0.0  0.0   
3   1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  6.0   0.0  ...  0.0  0.0  0.0   
4   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  ...  0.0  0.0  0.0   
..  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...  ...  ...  ...  ...   
95  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   2.0  ...  0.0  0.0  0.0   
96  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  5.0   4.0  ...  0.0  0.0  0.0   
97  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0   0.0  ...  0.0  0.0  0.0   
98  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   8.0  ...  0.0  0.0  0.0   
99  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  14.0  ...  0.0  0.0  0.0   

    777  778  779  780  781  782  783  
0   0.0  0.0  0.0  0.0  0.0  0.0  0

In [183]:
A0 = np.array(matrix_A['0'])
A0 = A0.T
Q0, R0 = np.linalg.qr(A0)

A1 = np.array(matrix_A['1'])
A1 = A1.T
Q1, R1 = np.linalg.qr(A1)

A2 = np.array(matrix_A['2'])
A2 = A2.T
Q2, R2 = np.linalg.qr(A2)

A3 = np.array(matrix_A['3'])
A3 = A3.T
Q3, R3 = np.linalg.qr(A3)

A4 = np.array(matrix_A['4'])
A4 = A4.T
Q4, R4 = np.linalg.qr(A4)

A5 = np.array(matrix_A['5'])
A5 = A5.T
Q5, R5 = np.linalg.qr(A5)

A6 = np.array(matrix_A['6'])
A6 = A6.T
Q6, R6 = np.linalg.qr(A6)

A7 = np.array(matrix_A['7'])
A7 = A7.T
Q7, R7 = np.linalg.qr(A7)

A8 = np.array(matrix_A['8'])
A8 = A8.T
Q8, R8 = np.linalg.qr(A8)

A9 = np.array(matrix_A['9'])
A9 = A9.T
Q9, R9 = np.linalg.qr(A9)

A = []
Q = []
R = []

for digit in range(0,10):
    A.append(np.array(matrix_A[f'{digit}']))
    A[digit] = A[digit].T
    Q_tmp,R_tmp = np.linalg.qr(A[digit])
    Q.append(Q_tmp)
    R.append(R_tmp)



In [ ]:
allTestFiles = os.listdir(PATH+f'/TinyMNIST/test/test/')
testlist = [PATH+f'/TinyMNIST/test/test/' + filename for
           filename in allTestFiles if filename[-4:] in [".jpg", ".JPG"]]
testDF = pd.read_csv(PATH+'/TinyMNIST/test labels.csv')
correctValues = 0
for testImage in testlist:
    imageId = re.sub("[^0-9]", "", testImage.split('/')[-1])
    npTest = numpy.array(Image.open(testImage), dtype=numpy.float)
    npTest = npTest.flatten()
    X=[]
    for digit in range(0,10):
        X.append(scipy.linalg.solve_triangular(R[digit], Q[digit].T.dot(npTest)))
    print(X[0].shape)
    ###Compute Mean Square
    dist = []
    for digit in range(0,10):
        dist.append(np.linalg.norm(npTest-(A[digit].dot(X[digit]))))
    print(dist)
    
    print('Image ID:',imageId)
    predictValue =(dist.index(min(dist)))
    print("Predict Value:",predictValue)
    print('RealValue:', int(testDF.loc[testDF['id']==int(imageId)]['category'].values))

    
    if int(predictValue)==int(testDF.loc[testDF['id']==int(imageId)]['category'].values):
        correctValues+=1
        print(True)
    else:
        print(False)
    print('---------------------------')
    

print(correctValues)
print(len(testDF))
print("Percent:",(correctValues/len(testDF))*100)

